<a href="https://colab.research.google.com/github/mori01-22/kennkyu/blob/main/Sample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/mori01-22/kennkyu/blob/main/Sample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# タンス検出 (二値分類) ノートブック

このノートブックは、与えられた画像（タンスあり / タンスなし）を識別するための学習・評価・推論パイプラインを示します。主に TensorFlow / Keras の転移学習（MobileNetV2）を利用します。

前提: 画像データはローカルに用意され、以下のようなフォルダ構成になっていることを想定します:

```
data/
  train/
    tansu/        # タンスが写っている画像（約1000枚程度）
    not_tansu/    # タンスが写っていない画像
```

(検証は自動で分割します)

ノート: このノートブックは Colab / ローカルどちらでも動きます。ローカル環境で GPU を使う場合は適切に TensorFlow をインストールしてください。

In [73]:
# 基本的なインポートと設定
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print('TensorFlow version:', tf.__version__)

# 再現性のためのシード設定（任意）
SEED = 123
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [74]:
# データセットのパスとパラメータ（必要に応じて変更）
data_dir = Path('data/train')  # ここに tansu と not_tansu のフォルダがある想定
img_size = (224, 224)
batch_size = 32

# ヘルパ: デモ用の簡易画像データを生成（data/demo_train に作成）
def create_demo_data(base_dir, img_size=(224,224), n_per_class=12):
    from PIL import Image, ImageDraw
    base = Path(base_dir)
    (base / 'tansu').mkdir(parents=True, exist_ok=True)
    (base / 'not_tansu').mkdir(parents=True, exist_ok=True)
    w, h = img_size
    for i in range(n_per_class):
        # タンスっぽい四角を描いた画像
        img = Image.new('RGB', (w, h), color=(150, 120, 90))
        d = ImageDraw.Draw(img)
        d.rectangle([int(w*0.2), int(h*0.2), int(w*0.8), int(h*0.9)], fill=(80, 60, 40))
        img.save(base / 'tansu' / f'tansu_{i}.jpg', 'JPEG')
        # タンスなし（丸）画像
        img2 = Image.new('RGB', (w, h), color=(200, 220, 240))
        d2 = ImageDraw.Draw(img2)
        d2.ellipse([int(w*0.3), int(h*0.3), int(w*0.7), int(h*0.7)], fill=(120, 140, 160))
        img2.save(base / 'not_tansu' / f'not_{i}.jpg', 'JPEG')
    print(f'Created demo dataset at: {base.resolve()} (each class: {n_per_class} images)')

# フォルダ存在確認・デモ作成（不足時）
if not data_dir.exists() or not any(p.is_dir() for p in data_dir.iterdir()):
    print('警告: data/train が見つからない、またはクラスフォルダが不足しています。デモ用データを生成します。')
    demo_dir = Path('data/demo_train')
    if not demo_dir.exists() or not any(p.is_dir() for p in demo_dir.iterdir()):
        create_demo_data(demo_dir, img_size=img_size, n_per_class=12)
    data_dir = demo_dir

# クラスフォルダ数の確認（label_mode='binary' の場合は 2 クラス必須）
subdirs = [p.name for p in data_dir.iterdir() if p.is_dir()]
print('Found class folders:', subdirs)
if len(subdirs) < 2:
    raise ValueError(f'二値分類には少なくとも2つのクラスフォルダが必要です。見つかったクラス: {subdirs}')

# データ読み込み (80% training, 20% validation) - TensorFlow のユーティリティを使用
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset='training',
    seed=SEED,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='binary',
    validation_split=0.2,
    subset='validation',
    seed=SEED,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print('Classes:', class_names)


In [75]:
# パフォーマンス最適化: キャッシュとプリフェッチ
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# サンプル画像を表示して確認
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[int(labels[i])])
        plt.axis('off')
plt.show()

In [76]:
0.05

In [77]:
# モデル構築: 転移学習 (EfficientNetB0 を優先) + 強化したヘッド
try:
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=img_size + (3,),
        include_top=False,
        weights='imagenet')
    print('Using EfficientNetB0 as backbone')
except Exception:
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=img_size + (3,),
        include_top=False,
        weights='imagenet')
    print('EfficientNet not available; using MobileNetV2 as backbone')

base_model.trainable = False  # まずは凍結してヘッドのみ学習

inputs = keras.Input(shape=img_size + (3,))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

# ラベルスムージングを使った損失関数（過学習を抑える）
loss_fn = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.03)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              loss=loss_fn,
              metrics=['accuracy'])

model.summary()

In [78]:
# コールバック: 早期終了、ReduceLROnPlateau、TensorBoard、モデル保存
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7),
    keras.callbacks.ModelCheckpoint('best_tansu_model.h5', save_best_only=True),
    keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=1)
]

# MixUp の実装（バッチ内でシャッフルして混合）
import tensorflow as _tf
def _sample_beta(alpha, shape):
    # approximate Beta by gamma sampling
    x = _tf.random.gamma(shape, alpha)
    y = _tf.random.gamma(shape, alpha)
    return x / (x + y)

def mixup_batch(images, labels, alpha=0.12):
    batch_size = _tf.shape(images)[0]
    lam = _sample_beta(alpha, [batch_size])
    lam_x = _tf.reshape(lam, [-1,1,1,1])
    lam_y = _tf.reshape(lam, [-1,1])
    idx = _tf.random.shuffle(_tf.range(batch_size))
    images2 = _tf.gather(images, idx)
    labels2 = _tf.gather(labels, idx)
    mixed_x = images * lam_x + images2 * (1 - lam_x)
    mixed_y = labels * lam_y + labels2 * (1 - lam_y)
    return mixed_x, mixed_y

# 訓練用データに MixUp を適用（任意、GPU での学習が推奨）
try:
    mixup_alpha = 0.12
    train_ds_mix = train_ds.map(lambda x,y: mixup_batch(x, tf.expand_dims(y, -1), alpha=mixup_alpha),
                                num_parallel_calls=AUTOTUNE)
    train_ds_mix = train_ds_mix.prefetch(AUTOTUNE)
    ds_to_train = train_ds_mix
    print('Applied MixUp to train dataset (alpha=', mixup_alpha, ')')
except Exception as e:
    print('Could not apply MixUp, falling back to original dataset:', e)
    ds_to_train = train_ds

# クラス不均衡対応: data_dir 配下を数えて class_weight を自動計算
try:
    counts = {}
    for cls in class_names:
        p = Path(data_dir)/cls
        counts[cls] = len(list(p.glob('*'))) if p.exists() else 0
    print('Class counts:', counts)
    total = sum(counts.values())
    n_classes = len(class_names)
    class_weight = {}
    for i, cls in enumerate(class_names):
        cnt = counts[cls]
        class_weight[i] = (total / (n_classes * cnt)) if cnt > 0 else 1.0
    print('Computed class_weight:', class_weight)
except Exception as e:
    print('Could not compute class weights from data_dir:', e)
    class_weight = None

# 学習（ヘッドのみ）
epochs = 12
history = model.fit(ds_to_train, validation_data=val_ds, epochs=epochs, callbacks=callbacks, class_weight=class_weight)

# --- 自動ファインチューニング: ベースモデルの一部を凍結解除して微調整 ---
try:
    print('Starting fine-tuning: unfreezing top layers of the base model...')
    base_model.trainable = True
    # fine_tune_at を調整してどの層から凍結解除するか決める
    fine_tune_at = 100
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    # 再コンパイル: 学習率はさらに小さく設定
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                  loss=loss_fn,
                  metrics=['accuracy'])

    fine_epochs = 6
    total_epochs = epochs + fine_epochs
    fine_callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
        keras.callbacks.ModelCheckpoint('best_tansu_model_finetuned.h5', save_best_only=True),
        keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-7)
    ]

    fine_history = model.fit(train_ds,
                             validation_data=val_ds,
                             epochs=total_epochs,
                             initial_epoch=epochs,
                             callbacks=fine_callbacks,
                             class_weight=class_weight)
except Exception as e:
    print('Fine-tuning skipped or failed:', e)

In [79]:
# モデル保存 — Keras ネイティブ形式（推奨）と SavedModel（TFLite/TF Serving 用）を両方試す
try:
    # Keras ネイティブ形式（単一ファイル）
    model.save('tansu_detector.keras')
    print('モデルを tansu_detector.keras に保存しました (Keras native format)')
except Exception as e:
    print('Keras native save failed:', e)
